In [3]:
import numpy as np

In [4]:
n_rolls = 10**7
attack_rolls  = np.random.randint(1, 7, size=n_rolls*3).reshape(n_rolls,3)
defense_rolls = np.random.randint(1, 7, size=n_rolls*2).reshape(n_rolls,2)

In [5]:
defense_rolls.sort(axis=1)
attack_rolls.sort(axis=1)

In [6]:
mod_attack_1 = 0
mod_attack_2 = 0
mod_defense_1 = 1
mod_defense_2 = 1

# Rolls need to be modified but can only be a max value of 6.
best_attack_rolls = np.minimum((attack_rolls[:,-1] + mod_attack_1), 6)
second_best_attack_rolls = np.minimum((attack_rolls[:,-2] + mod_attack_2), 6)

best_defense_rolls = np.minimum((defense_rolls[:,-1] + mod_defense_1), 6)
second_best_defense_rolls = np.minimum((defense_rolls[:,-2] + mod_defense_2), 6)


In [7]:
defenders_lost = (np.sum(best_attack_rolls > best_defense_rolls) +
                  np.sum(second_best_attack_rolls > second_best_defense_rolls))

attackers_lost = (np.sum(best_attack_rolls <= best_defense_rolls) +
                  np.sum(second_best_attack_rolls <= second_best_defense_rolls))

In [8]:
(defenders_lost/attackers_lost)**-1

1.8936503906066322

In [9]:
def simulate_single_battle(n_attackers=3, n_defenders=2,
                           modifiers={'attack1':0, 'attack2':0, 'defense1':0, 'defense2':0}):
    attackers_killed = 0
    defenders_killed = 0
    
    attack_roll  = np.random.randint(1, 7, size=min(n_attackers, 3))
    defense_roll = np.random.randint(1, 7, size=min(n_defenders, 2))

    attack_roll.sort()
    defense_roll.sort()
    
    modified_attack_roll1  = np.minimum((attack_roll[-1] + modifiers['attack1']), 6)
    modified_defense_roll1 = np.minimum((defense_roll[-1] + modifiers['defense1']), 6)
    
    # Compare the highest roll of the attacker and defender.
    if modified_attack_roll1 > modified_defense_roll1:
        defenders_killed += 1
    else:
        attackers_killed += 1
        
    # Compare 2nd highest die rolls if both attacker and defender used more than one die.
    if n_attackers > 1 and n_defenders > 1:
        modified_attack_roll2 = np.minimum((attack_roll[-2] + modifiers['attack2']), 6)
        modified_defense_roll2 = np.minimum((defense_roll[-2] + modifiers['defense2']), 6)
        
        if modified_attack_roll2 > modified_defense_roll2:
            defenders_killed += 1
        else:
            attackers_killed += 1
        
    n_attackers_remaining = n_attackers - attackers_killed
    n_defenders_remaining = n_defenders - defenders_killed
        
    return n_attackers_remaining, n_defenders_remaining

In [10]:
simulate_single_battle(15, 10)

(13, 10)

In [11]:
wins = {'attacker':0, 'defender':0}

for _ in range(10):

    n_attackers = 10
    n_defenders = 10


    while n_attackers > 0 and n_defenders > 0:
        n_attackers, n_defenders = simulate_single_battle(n_attackers, n_defenders)

    if n_attackers > 0:
        wins['attacker'] += 1
    else:
        wins['defender'] += 1

wins

{'attacker': 6, 'defender': 4}

In [12]:
def simulate_n_battles(n_attackers, n_defenders, n_simulations):
    wins = {'attacker':0, 'defender':0}
    
    for _ in range(n_simulations):
        n_attack = n_attackers
        n_defend = n_defenders
        
        while n_attack > 0 and n_defend > 0:
            n_attack, n_defend = simulate_single_battle(n_attack, n_defend)

        if n_attack > 0:
            wins['attacker'] += 1
        else:
            wins['defender'] += 1
    
    return wins

In [13]:
simulate_n_battles(35, 20, 10**4)

{'attacker': 9876, 'defender': 124}

In [106]:
x = np.array([1,2,5])
y = np.array([2,3,4])

In [107]:
import random

In [108]:
for i in range(0):
    print(i)

In [109]:
# .25 - 0
# .25 - 1
# .50 - 2
n_trials = 10000
successes = 0

for _ in range(n_trials):
    amoebas = [1]
    n_iterations = 0
    while amoebas and n_iterations < 1000:
        n_iterations += 1
        amoebas.pop()
        for _ in range(generate_children()):
            amoebas.append(1)
            
    if amoebas:
        successes += 1

print((n_trials - successes)/n_trials)

0.5003


In [110]:
def generate_children():
    num = random.random()
    if num <= .25:
        children = 0
    elif num <= .50:
        children = 1
    else:
        children = 2
    return children

In [111]:
generate_children()

2